In [1]:
import os
import sys
import Wind_KAN_
import importlib
import time
import matplotlib.pyplot as plt

In [2]:
from Wind_KAN_ import *
from kan import *
torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
winddata_path = os.path.join('..\\Liao_code','Task15_W_Zone6.csv')
train_val_split = 0.1
train_test_split = 0.1
out=False

data = pd.read_csv(winddata_path)
Y = data.iloc[:, 2].values.astype(np.float64)
X = data.iloc[:, 3:7].values.astype(np.float64)

scaler = MinMaxScaler()
X= scaler.fit_transform(X)
split1 = int((1-train_test_split) * train_val_split * len(X))
split2 = int((1-train_test_split) * len(X))

# for comparison with other models
split1 = int(0.7 * len(X))
split2 = int(0.85 * len(X))
##########################################

X_train, y_train = X[:split1], Y[:split1]
X_val, y_val = X[split1:split2], Y[split1:split2]
X_test, y_test = X[split2:], Y[split2:]

ndarray_data = [X_train, y_train, X_val, y_val, X_test, y_test]
tensor_data_list = [torch.from_numpy(x) for x in ndarray_data]

tensor_data_list[1] = tensor_data_list[1].unsqueeze(1)  # y_train
tensor_data_list[3] = tensor_data_list[3].unsqueeze(1)  # y_val
tensor_data_list[5] = tensor_data_list[5].unsqueeze(1)  # y_test

# The KAN Class trains on train/validation called train_input/test_input
# dataset = { 'train_input': tensor_data_list[0].float(),
#             'train_label': tensor_data_list[1].float(),
#             'test_input': tensor_data_list[2].float(),
#             'test_label': tensor_data_list[3].float(),
#             'true_test_input': tensor_data_list[4].float(),
#             'true_test_label': tensor_data_list[5].float()  }

dataset = { 'train_input': tensor_data_list[0],
            'train_label': tensor_data_list[1],
            'test_input': tensor_data_list[2],
            'test_label': tensor_data_list[3],
            'true_test_input': tensor_data_list[4],
            'true_test_label': tensor_data_list[5]  }

In [4]:
from kan.utils import create_dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# create dataset f(x,y) = exp(sin(pi*x)+y^2)
f = lambda x: torch.exp(torch.sin(torch.pi*x[:,[0]]) + x[:,[1]]**2)
hello_dataset = create_dataset(f, n_var=2, device=device)

# for i, key in enumerate(hello_dataset.keys()):
#     hello_dataset[key] = hello_dataset[key].float()

In [5]:
print(hello_dataset['train_input'][0])
print(type(hello_dataset['train_input'][0]))
print(dataset['train_input'][0])
print(type(dataset['train_input'][0]))

tensor([0.9401, 0.9797])
<class 'torch.Tensor'>
tensor([0.4335, 0.4534, 0.4360, 0.4929])
<class 'torch.Tensor'>


In [12]:
model = KAN(width=[4,1], grid=3, k=3, seed=42, device=device)

checkpoint directory created: ./model
saving model version 0.0


In [13]:
results = model.fit(dataset, opt="LBFGS", steps=50, lamb=0.001)

| train_loss: 1.97e-01 | test_loss: 1.90e-01 | reg: 4.16e+00 | : 100%|█| 50/50 [00:28<00:00,  1.76it

saving model version 0.1


In [8]:
hello_model = KAN(width=[2,5,1], grid=3, k=3, seed=42, device=device)

checkpoint directory created: ./model
saving model version 0.0


In [9]:
results = hello_model.fit(hello_dataset)

| train_loss: 1.31e-03 | test_loss: 1.32e-03 | reg: 1.80e+01 | : 100%|█| 100/100 [00:45<00:00,  2.20

saving model version 0.1


In [14]:
model = model.prune()

saving model version 0.2


In [4]:
wind_model = Wind_KAN(width=[4,1], grid=3, k=3, seed=42, device=device)
wind_model.get_winddataset(winddata_path)

checkpoint directory created: ./model
saving model version 0.0


In [10]:
import inspect
lines = inspect.getsource(wind_model.get_winddataset)
print(lines)

    def get_winddataset(self, winddata_path, train_val_split = 0.1, train_test_split = 0.1, out=False):
        self.winddata_path = winddata_path
        data = pd.read_csv(self.winddata_path)
        Y = data.iloc[:, 2].values.astype(np.float64)
        X = data.iloc[:, 3:7].values.astype(np.float64)

        scaler = MinMaxScaler()
        X= scaler.fit_transform(X)
        split1 = int((1-train_test_split) * train_val_split * len(X))
        split2 = int((1-train_test_split) * len(X))

        # for comparison with other models
        split1 = int(0.7 * len(X))
        split2 = int(0.85 * len(X))
        ##########################################

        X_train, y_train = X[:split1], Y[:split1]
        X_val, y_val = X[split1:split2], Y[split1:split2]
        X_test, y_test = X[split2:], Y[split2:]

        ndarray_data = [X_train, y_train, X_val, y_val, X_test, y_test]
        tensor_data_list = [torch.from_numpy(x) for x in ndarray_data]

        tensor_data_list[1] = tensor_d

In [6]:
results = wind_model.fit_model()

| train_loss: 1.90e-01 | test_loss: 1.88e-01 | reg: 8.57e+00 | : 100%|█| 20/20 [00:07<00:00,  2.52it

saving model version 0.1


In [8]:
results = wind_model.prune_model()

saving model version 0.2


| train_loss: 1.89e-01 | test_loss: 1.89e-01 | reg: 8.59e+00 | : 100%|█| 20/20 [00:09<00:00,  2.03it

saving model version 0.2
